# Power Flow Calculations Without Main Field Losses

The test bench consists of three nodes, a three winding transformer and two loads at the medium and low voltage port of
the transformer. We sweep through tap positions and active power consumption of the loads, respectively.
This is meant to get an insight into the results of the transformer, that are achieved with pandapower.

As the power rating of the high voltage port is 300 MVA, that of the medium voltage port 300 MVA as well and 100 MVA for
the low voltage port, the following applies:
The medium voltage load is varied within the range of +/- 300 MVA.
The low voltage load is only varied within the permissible range, that is left.
E.g. if the medium voltage load is 250 MVA, ist only is varied from -100 MVA to 50 MVA.

In [ ]:
# Specify information about persisted entities
import json
import os

from calculation.pandapower.ThreeWindingTestBench import ThreeWindingTestBench
from encoder import CustomDecoder
from encoder.DictEncoder import DictEncoder

result_directory = os.path.join("..", "..", "results", "three_winding")
result_file = os.path.join(result_directory, "withoutMainFieldLosses.json")

# Define basic parameters
s_rated_hv_mva: float = 300
s_rated_mv_mva: float = 300
s_rated_lv_mva: float = 100
v_ref_kv: float = 380
s_ref_mva: float = 300
tap_min: int = -10
tap_max: int = 10
p_step_size: float = 0.1  # in p.u.

if os.path.exists(result_file):
    # The result file exists. Read it and use it
    print("Pre-calculated results are available. Load them.")
    with open(result_file, "r") as file_to_read:
        json_string = file_to_read.read()
        results = json.loads(json_string, object_hook=CustomDecoder.custom_decode)
else:
    # Result file doesn't exist. Perform simulations and write the file
    results: list = ThreeWindingTestBench().calculate(tap_min=tap_min, tap_max=tap_max, s_nom_hv_mva=s_rated_hv_mva,
                                                      s_nom_mv_mva=s_rated_mv_mva, s_nom_lv_mva=s_rated_lv_mva,
                                                      p_step_pu=p_step_size, v_ref_kv=v_ref_kv, s_ref_mva=s_ref_mva,
                                                      tap_at_star_point=False)

    if not os.path.exists(result_directory):
        os.makedirs(result_directory)
    with open(result_file, "w") as file_to_write_to:
        json.dump(results, file_to_write_to, cls=DictEncoder, indent=2)

### Plot the results

#### Nodal voltages in neutral tap position

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

# --- Extract all information from results ---
p_mv_mw = []  # Power at medium voltage node
p_lv_mw = []  # Power at low voltage node
v_mv_pu = []  # Voltage magnitude at medium voltage node
v_lv_pu = []  # Voltage magnitude at low voltage node
p_hv_mw = []  # Active power at high voltage node
q_hv_mvar = []  # Reactive power at high voltage node
for entry in filter(lambda result: result['tap_pos'] == 0, results):
    p_mv_mw.append(entry['p_mv'])
    p_lv_mw.append(entry['p_lv'])
    v_mv_pu.append(entry['result'].v_mv_pu)
    v_lv_pu.append(entry['result'].v_lv_pu)
    p_hv_mw.append(entry['result'].p_hv_kw / 1000)
    q_hv_mvar.append(entry['result'].q_hv_kvar / 1000)

v_min = min(v_mv_pu + v_lv_pu)
v_max = max(v_mv_pu + v_lv_pu)
print("Lowest voltage: %.3f p.u., highest voltage: %.3f p.u." % (v_min, v_max))

fig_neutral_tap_v = plt.figure(figsize=(25,10))
cmap_neutral_tap = plt.get_cmap('hot')
ax_neutral_tap_v_mv = fig_neutral_tap_v.add_subplot(1, 2, 1, projection='3d')
ax_neutral_tap_v_mv.set_title('Voltage magnitude - medium voltage port')
ax_neutral_tap_v_mv.set_xlabel('Active power at medium voltage port in MW')
ax_neutral_tap_v_mv.set_ylabel('Active power at low voltage port in MW')
ax_neutral_tap_v_mv.set_zlabel('Voltage magnitude at medium voltage port in p.u.')
ax_neutral_tap_v_mv.set_zlim(v_min, v_max)
ax_neutral_tap_v_mv.plot_trisurf(np.array(p_mv_mw), np.array(p_lv_mw), np.array(v_mv_pu), cmap=cmap_neutral_tap)

ax_neutral_tap_v_lv = fig_neutral_tap_v.add_subplot(1, 2, 2, projection='3d')
ax_neutral_tap_v_lv.set_title('Voltage magnitude - low voltage port')
ax_neutral_tap_v_lv.set_xlabel('Active power at medium voltage port in MW')
ax_neutral_tap_v_lv.set_ylabel('Active power at low voltage port in MW')
ax_neutral_tap_v_lv.set_zlabel('Voltage magnitude at low voltage port in p.u.')
ax_neutral_tap_v_lv.set_zlim(v_min, v_max)
ax_neutral_tap_v_lv.plot_trisurf(np.array(p_mv_mw), np.array(p_lv_mw), np.array(v_lv_pu), cmap=cmap_neutral_tap)

plt.show()

##### Conclusions
-   The higher the power consumption, the lower the nodal voltages.
-   Power consumption at one port also slightly affects the nodal voltage magnitude at the other port.
-   When only varying the power consumption at the low voltage port:
    Voltage magnitude at low voltage port varies more than on medium voltage port.
    Therefore, Main part of the impedance has to be between star point and low voltage port.
-   Same applies for the impedance between star point and medium voltage port.
-   Nodal voltage is 1.0 p.u. at both ports for the no-load situation.
    This meets expectations for a transformer without main field losses.

##### Open questions
-   Why isn't the nodal voltage magnitude higher, when feeding in power? Let's have a look at the high voltage port power

#### High voltage port power in neutral tap position

In [ ]:
fig_neutral_tap_s_hv = plt.figure(figsize=(25,10))
ax_neutral_tap_p_hv = fig_neutral_tap_s_hv.add_subplot(1, 2, 1, projection='3d')
ax_neutral_tap_p_hv.set_title('Power at high voltage node - Active power')
ax_neutral_tap_p_hv.set_xlabel('Active power at medium voltage port in MW')
ax_neutral_tap_p_hv.set_ylabel('Active power at low voltage port in MW')
ax_neutral_tap_p_hv.set_zlabel('Active power at high voltage node in MW')
ax_neutral_tap_p_hv.plot_trisurf(np.array(p_mv_mw), np.array(p_lv_mw), np.array(p_hv_mw), cmap=cmap_neutral_tap)

ax_neutral_tap_q_hv = fig_neutral_tap_s_hv.add_subplot(1, 2, 2, projection='3d')
ax_neutral_tap_q_hv.set_title('Power at high voltage node - Reactive power')
ax_neutral_tap_q_hv.set_xlabel('Active power at medium voltage port in MW')
ax_neutral_tap_q_hv.set_ylabel('Active power at low voltage port in MW')
ax_neutral_tap_q_hv.set_zlabel('Reactive power at high voltage node in MVAr')
ax_neutral_tap_q_hv.plot_trisurf(np.array(p_mv_mw), np.array(p_lv_mw), np.array(q_hv_mvar), cmap=cmap_neutral_tap)

##### Conclusions
-   Active power seems to sum up as expected, taking resistive losses into account
-   Reactive power is purely inductive and increases with total transferred power